In [ ]:
%reload_ext sql

In [1]:
import pandas as pd
import boto3
from botocore.exceptions import ClientError
import json
import configparser
import psycopg2
from sqlalchemy import create_engine
 

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('/Users/andrekrasinski/Documents/GitHub/DataEngineerAWS/dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')
REGION                 = config.get('AWS','REGION')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

DWH_ENDPOINT=config.get("DWH", "DWH_ENDPOINT")
DWH_ROLE_ARN=config.get("DWH", "DWH_ROLE_ARN")


(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

In [3]:
#Create clients for each service
ec2 = boto3.resource('ec2', region_name=REGION, aws_access_key_id=KEY, aws_secret_access_key=SECRET)

s3 = boto3.resource('s3', region_name=REGION, aws_access_key_id=KEY, aws_secret_access_key=SECRET)

iam = boto3.client('iam', region_name=REGION, aws_access_key_id=KEY, aws_secret_access_key=SECRET)

redshift = boto3.client('redshift', region_name=REGION, aws_access_key_id=KEY, aws_secret_access_key=SECRET)

In [ ]:
#Run over the sample data bucket
sampleDbBucket =  s3.Bucket("krasinski-udacity-1")

for file in sampleDbBucket.objects.all():
    print(file)

In [ ]:
#Create a IAM role that makes Redshift able to access S3
try:
    print("1.1 Creating a new IAM Role")
    dwhRole = iam.create_role(
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps({
            'Statement': [{'Action': 'sts:AssumeRole',
                           'Effect': 'Allow',
                           'Principal': {'Service': 'redshift.amazonaws.com'}}],
            'Version': '2012-10-17'
        })
    )
except Exception as e:
    print(e)

In [ ]:
#Attach POlicy
try:
    print("1.2 Attaching Policy")
    iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")['ResponseMetadata']['HTTPStatusCode']
except Exception as e:
    print(e)
    

In [ ]:
#get and print the role ARN
try:
    print("1.3 Get the role ARN")
    roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
    print(roleArn)
except Exception as e:
    print(e)
    
    

In [ ]:
#Create a redshift cluster
try:
    print("1.4 Creating a redshift cluster")
    response = redshift.create_cluster(
        #Parameters for the hardware configuration of the cluster
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),
        #add
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        Port=int(DWH_PORT),
        #Parameters for the IAM roles and other security settings
        IamRoles=[roleArn]
    )
    print(response)
except Exception as e:
    print(e)

In [ ]:
#describe the cluster to see its status
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', 1000)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
#take note of the endpoint and role arn
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

In [ ]:
#Open an incoming TCP port to access the cluster
try:
    print("1.5 Open an incoming TCP port to access the cluster")
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        GroupName=defaultSg.group_name,
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)


In [ ]:

conn = psycopg2.connect(
    host=DWH_ENDPOINT,
    dbname=DWH_DB,
    user=DWH_DB_USER,
    password=DWH_DB_PASSWORD,
    port=DWH_PORT
)
print("Connection successful!")

cur = conn.cursor()

def execute_query(query, params=None):
    try:
        if params:
            cur.execute(query, params)
        else:
            cur.execute(query)
        conn.commit()
        print("Query executed successfully")
        return cur.fetchall()
    except Exception as e:
        print(f"An error occurred: {e}")
        conn.rollback()
        return None


In [ ]:
query = 'SELECT 1'
result = execute_query(query)
print (result)

cur.close()
conn.close()


In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)